# Algorithms in Bioinformatics / Homework 3

1) Write a program to build an alignment using the Smith-Waterman algorithm.

The penalties for insertions-deletions are linear (not affine), the parameters are: the reward for
match m=1, the penalty for replacement mism=1, the penalty for deletion d=1. 

The program should
output the alignment (sequences that are written under
each other) and the alignment score.

In [ ]:
import pandas as pd
import itertools

let's create the matrix:

In [ ]:
def SmithWaterman(seq1,seq2,m,mism,d):
    matrix = pd.DataFrame(index=["0"]+[n for n in seq1], columns=["0"]+[n for n in seq2]) # create an empty matrix
    for i, j in itertools.product(range(len(matrix.index)),range(len(matrix.columns))): # it goes row by row
        # both the first row and column are filled w/ 0
        if i*j == 0:
            matrix.iloc[i,j] = 0
        else:
            # the values for three previous cells
            cell_1 = matrix.iloc[i,j-1]
            cell_2 = matrix.iloc[i-1,j-1]
            cell_3 = matrix.iloc[i-1,j]
            if seq1[i-1] == seq2[j-1]:
                matrix.iloc[i,j] = max(0, cell_1-d, cell_2+m, cell_3-d) # choose the maxim. taking into account the penalties for deletions and the rewards for matching
            else:
                matrix.iloc[i,j]=max(0, cell_1-d, cell_2-mism, cell_3-d) # choose the maxim. taking into account the penalties for deletions and mismatches
    print(matrix)

In [ ]:
SmithWaterman("AAACGTTGC","TGCACGA",1,1,1)

   0  T  G  C  A  C  G  A
0  0  0  0  0  0  0  0  0
A  0  0  0  0  1  0  0  1
A  0  0  0  0  1  0  0  1
A  0  0  0  0  1  0  0  1
C  0  0  0  1  0  2  1  0
G  0  0  1  0  0  1  3  2
T  0  1  0  0  0  0  2  2
T  0  1  0  0  0  0  1  1
G  0  0  2  1  0  0  1  0
C  0  0  1  3  2  1  0  0


In [ ]:
def SmithWaterman(seq1,seq2,m,mism,d):
    matrix = pd.DataFrame(index=["0"]+[n for n in seq1], columns=["0"]+[n for n in seq2]) # create an empty matrix
    for i, j in itertools.product(range(len(matrix.index)),range(len(matrix.columns))): # it goes row by row
        # both the first row and column are filled w/ 0
        if i*j == 0:
            matrix.iloc[i,j] = 0
        else:
            # the values for three previous cells
            cell_1 = matrix.iloc[i,j-1]
            cell_2 = matrix.iloc[i-1,j-1]
            cell_3 = matrix.iloc[i-1,j]
            if seq1[i-1] == seq2[j-1]:
                matrix.iloc[i,j] = max(0, cell_1-d, cell_2+m, cell_3-d) # choose the maxim. taking into account the penalties for deletions and the rewards for matching
            else:
                matrix.iloc[i,j]=max(0, cell_1-d, cell_2-mism, cell_3-d) # choose the maxim. taking into account the penalties for deletions and mismatches
    #print(matrix) # we start then from its end
    # create aligned sequences (that are empty now)
    aligned_seq1 = ""
    aligned_seq2 = ""
    # find the max. element in the last row/column and start w/ it  
    max_elem1 = matrix.iloc[:,len(seq2)].max() 
    max_elem2 = matrix.iloc[len(seq1)].max()
    if max_elem1 > max_elem2: 
        i = len(seq1)
        j = len(seq2)
        while True:
          if matrix.iloc[i,len(seq2)] == max_elem1:
            aligned_seq1 = aligned_seq1+matrix.index[i]
            aligned_seq2 = aligned_seq2+matrix.columns[len(seq2)]
            break
          else:
            aligned_seq1 = aligned_seq1+matrix.index[i]
            aligned_seq2 = aligned_seq2+"-"
            i = i-1
    else:
      i = len(seq1)
      j = len(seq2)
      while True:
        if matrix.iloc[len(seq1),j] == max_elem1:
          aligned_seq1 = aligned_seq1+matrix.index[len(seq1)]
          aligned_seq2 = aligned_seq2+matrix.columns[j]
          break
        else:
          aligned_seq1 = aligned_seq1+"-"
          aligned_seq2 = aligned_seq2+matrix.columns[j]
          j=j-1
    for i,j in itertools.product(range(len(matrix.index)),range(len(matrix.columns))):
      if i*j == 0:
        matrix.iloc[i,j] = -1 # both the first row and column are filled with -1 (not to go beyond the boundaries)
   # go to one of the previous cell w/ the max value
    while i>1 or j>1:
      cell_1 = matrix.iloc[i,j-1]
      cell_2 = matrix.iloc[i-1,j-1]
      cell_3 = matrix.iloc[i-1, j]
      max_s = max(cell_1,cell_2,cell_3)
      if cell_1 == max_s:
        j=j-1
        aligned_seq1 = aligned_seq1+"-"
        aligned_seq2 = aligned_seq2+matrix.columns[j]
      elif cell_2 == max_s:
        i=i-1
        j=j-1
        aligned_seq1 = aligned_seq1+matrix.index[i]
        aligned_seq2 = aligned_seq2+matrix.columns[j]
      else:
        i=i-1
        aligned_seq1 = aligned_seq1+matrix.index[i]
        aligned_seq2 = aligned_seq2+"-"
    print(aligned_seq1[::-1])
    print(aligned_seq2[::-1])
    print("Score:", max(max_elem1,max_elem2))

2) To test the program on several examples:

Since a local alignment is being sought, there should be such cases when the beginnings and/or endings of the sequences are not the same (dif).

In [ ]:
SmithWaterman("TAATCTAGA","TAATCTAGA",1,1,1) # when the sequences match

TAATCTAGA
TAATCTAGA
Score: 9


In [ ]:
SmithWaterman("TATTAGACGA","AATAGACC",1,1,1) # when the sequences have indels (the dif. start and end between each other)

T-ATTAGACGGA
-AA-TAGA-CC-
Score: 4


In [ ]:
SmithWaterman("ATTACGACCCGG","ATACGACGAGA",1,1,1) # when the sequences have indels (the end is not the same)

ATTACGACCC--G-G
A-TACGA--CGA-GA
Score: 5


In [ ]:
SmithWaterman("GATTAGA","ATTGA",1,1,1) # when the sequences have indels (the start is not the same) 
# this is also the sample of substitution "A-T"

GATTAGA
-AT-TGA
Score: 4


3*) To investigate how is changed the alignment weight of random independent sequences according to their length.

Let's look at first the cases when we know the input seqs similarity (they are identical or not):
1. If the sequences are the same, then the alignment weight is proportional to the length: W ~ L. if the sequences are the same, then any next addition will increase the alignment weight by 1.
2. If the sequences are not the same, then the alignment weight is less than the length: W < L.


For any sequences, this is true that: W(S1S2) <= L, 

where S1,2 - sequences; 

and it is also true that the alignment weight is enclosed between the two linear functions: CL <= W(S1S2) <= L (where C - the constant (~ on a model).

From this, we can conclude that **the alignment weight linearly depends on the length of the input sequences**, regardless of the model used.